In [1]:
## 參考學習 https://blog.csdn.net/xingchenbingbuyu/article/details/70169665
import glob as gb
import cv2
from matplotlib import pyplot as plt

In [3]:
# -*- coding: UTF-8 -*-
img = cv2.imread('sudoku.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
## 阈值分割
ret,thresh = cv2.threshold(gray,200,255,1)

## 对二值图像执行膨胀操作
kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(5, 5))     
dilated = cv2.dilate(thresh,kernel)

## 轮廓提取，cv2.RETR_TREE表示建立层级结构
_ ,contours, hierarchy = cv2.findContours(dilated,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

## 提取小方格，其父轮廓都为0号轮廓
boxes = []
for i in range(len(hierarchy[0])):
    if hierarchy[0][i][3] == 0:
        boxes.append(hierarchy[0][i])

## 提取数字，其父轮廓都存在子轮廓        
number_boxes = []
for j in range(len(boxes)):
    if boxes[j][2] != -1:
        #number_boxes.append(boxes[j])
        x,y,w,h = cv2.boundingRect(contours[boxes[j][2]]) ##矩形邊框（Bounding Rectangle）是說，用一個最小的矩形，把找到的形狀包起來。還有一個帶旋轉的矩形，面積會更小
        number_boxes.append([x,y,w,h])
        img = cv2.rectangle(img,(x,y),(x+w+1,y+h+1),(0,0,255),2)


#  擷取圖片0918~0928

In [4]:
img_path = gb.glob("numbers\\*") 

k = 0
labels = []
samples =  []

##(一)提取數字  
##1.遍歷文件夾下的原始數字圖片；
##2.對每一張圖片進行輪廓提取操作，只提取外圍輪廓（參考上一節講解）；

for path in img_path:
    img  = cv2.imread(path)       
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),0)
    thresh = cv2.adaptiveThreshold(blur,255,1,1,11,2)    
    contours, hierarchy =  cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    height,width = img.shape[:2]
    w = width/5
    rect_list = []
    list1 = []
    list2 = []
    
##3.求輪廓外包矩形，並根據矩形大小信息篩選出所有的數字輪廓；

    for cnt in contours:
        #if cv2.contourArea(cnt)>100:
        [x,y,w,h] = cv2.boundingRect(cnt)

        if w>30 and h > (height/4):   
            if y < (height/2):
                list1.append([x,y,w,h])
##4.然後根據位置信息對數字框排序，顯然第一排依次是12345，第二排依次是67890；
            else:
                list2.append([x,y,w,h])
            
    ls1 = sorted(list1,key = lambda t : t[0])
    ls2 = sorted(list2,key = lambda t : t[0])
    

    if len(ls1) == 5 and len(ls2) == 5: 
        
    ##5.提取出每一個數字所在的矩形框，作為ROI取出。   
        for i in range(5):
            [x1,y1,w1,h1] = ls1[i] 
            [x2,y2,w2,h2] = ls2[i]                
            number_roi1 = gray[y1:y1+h1, x1:x1+w1] #Cut the frame to size
            number_roi2 = gray[y2:y2+h2, x2:x2+w2] #Cut the frame to size  
        ##數據預處理
            resized_roi1=cv2.resize(number_roi1,(20,40))
            thresh1 = cv2.adaptiveThreshold(resized_roi1,255,1,1,11,2)

            resized_roi2=cv2.resize(number_roi2,(20,40))
            thresh2 = cv2.adaptiveThreshold(resized_roi2,255,1,1,11,2)

        ##把二值圖像轉換為0-1二值圖像。
        ##把處理完的數字圖片保存到對應數字的文件夾中。
            number_path1 = "test\\%s\\%d" % (str(i+1),k) + '.jpg'
            j = i+6
            if j ==10:
                j = 0

            number_path2 = "test\\%s\\%d" % (str(j),k) + '.jpg'
            k+=1

            normalized_roi1 = thresh1/255.
            normalized_roi2 = thresh2/255.
            ## 把图片展开成一行，然后保存到samples
            ## 保存一个图片信息，保存一个对应的标签
            sample1 = normalized_roi1.reshape((1,800))
            samples.append(sample1[0])
            labels.append(float(i+1))

            sample2 = normalized_roi2.reshape((1,800))
            samples.append(sample2[0])
            labels.append(float(j))

            cv2.imwrite(number_path1,thresh1)
            cv2.imwrite(number_path2,thresh2)
            #cv2.imshow("number",normalized_roi1)
            #cv2.waitKey(5)
    else:
        print(path)

In [4]:
def findNextCellToFill(grid, i, j):
    for x in range(i,9):
        for y in range(j,9):
            if grid[x][y] == 0:
                return x,y
    for x in range(0,9):
        for y in range(0,9):
            if grid[x][y] == 0:
                return x,y
    return -1,-1

def isValid(grid, i, j, e):
    rowOk = all([e != grid[i][x] for x in range(9)])
    if rowOk:
        columnOk = all([e != grid[x][j] for x in range(9)])
        if columnOk:
            # finding the top left x,y co-ordinates of the section containing the i,j cell
            secTopX, secTopY = 3 *int(i/3), 3 *int(j/3)
            for x in range(secTopX, secTopX+3):
                for y in range(secTopY, secTopY+3):
                    if grid[x][y] == e:
                        return False
                return True
    return False

def solveSudoku(grid, i=0, j=0):
    i,j = findNextCellToFill(grid, i, j)
    if i == -1:
        return True
    for e in range(1,10):
        if isValid(grid,i,j,e):
            grid[i][j] = e
            if solveSudoku(grid, i, j):
                return True
            # Undo the current cell for backtracking
            grid[i][j] = 0
    return False


## 0929 解出方格

In [8]:
import numpy as np

samples = np.array(samples,np.float32)
#samples = samples.reshape((samples.size,1))
labels = np.array(labels,np.float32)
labels = labels.reshape((labels.size,1))

np.save('samples.npy',samples)
np.save('label.npy',labels)
 
## 訓練knn模型
samples = np.load('samples.npy')
labels = np.load('label.npy')

k = 80
train_label = labels[:k]
train_input = samples[:k]
test_input = samples[k:]
test_label = labels[k:]

model = cv2.ml.KNearest_create()
model.train(train_input,cv2.ml.ROW_SAMPLE,train_label)



img = cv2.imread('sudoku.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
## 阈值分割
ret,thresh = cv2.threshold(gray,200,255,1)

kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(5, 5))     
dilated = cv2.dilate(thresh,kernel)
 
## 輪廓提取
contours, hierarchy = cv2.findContours(dilated,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

##　提取八十一個小方格
boxes = []
for i in range(len(hierarchy[0])):
    if hierarchy[0][i][3] == 0:
        boxes.append(hierarchy[0][i])
        
height,width = img.shape[:2]
box_h = height/9
box_w = width/9
number_boxes = []
## 數獨初始化為零陣
soduko = np.zeros((9, 9),np.int32)

for j in range(len(boxes)):
    if boxes[j][2] != -1:
        #number_boxes.append(boxes[j])
        x,y,w,h = cv2.boundingRect(contours[boxes[j][2]])
        number_boxes.append([x,y,w,h])
        #img = cv2.rectangle(img,(x-1,y-1),(x+w+1,y+h+1),(0,0,255),2)
        #img = cv2.drawContours(img, contours, boxes[j][2], (0,255,0), 1)
        ## 提取數字預處理
        number_roi = gray[y:y+h, x:x+w]
        ## 统一大小
        resized_roi=cv2.resize(number_roi,(20,40))
        thresh1 = cv2.adaptiveThreshold(resized_roi,255,1,1,11,2) 
        ## 歸一化像素值
        normalized_roi = thresh1/255.  
        
        ## 展開成一行讓knn識別
        sample1 = normalized_roi.reshape((1,800))
        sample1 = np.array(sample1,np.float32)
        
        ## knn識別
        retval, results, neigh_resp, dists = model.findNearest(sample1, 1)        
        number = int(results.ravel()[0])
        
        ## 識別结果展示
        cv2.putText(img,str(number),(x+w+1,y+h-20), 3, 2., (255, 0, 0), 2, cv2.LINE_AA)
        
        ## 求在矩阵中的位置
        soduko[int(y/box_h)][int(x/box_w)] = number
               
        #print(number)
        

print("\n生成的數獨\n")
print(soduko)

print("\n求解后的数独\n")

## 数独求解

 


生成的數獨

[[7 6 0 0 5 0 0 8 0]
 [4 0 0 0 3 0 2 0 0]
 [0 3 2 0 0 0 4 0 6]
 [0 0 0 0 0 8 0 0 0]
 [0 0 9 0 6 0 8 0 0]
 [0 0 0 2 0 0 0 0 0]
 [6 0 8 0 0 0 9 1 0]
 [0 0 7 0 4 0 0 0 5]
 [0 2 0 0 9 0 0 3 8]]

求解后的数独



In [9]:
solveSudoku(soduko)

print(soduko)
print("\n验算：求每行每列的和\n")
row_sum = map(sum,soduko)
col_sum = map(sum,zip(*soduko))
print(list(row_sum))
print(list(col_sum))

[[7 6 1 4 5 2 3 8 9]
 [4 8 5 9 3 6 2 7 1]
 [9 3 2 1 8 7 4 5 6]
 [5 4 6 3 7 8 1 9 2]
 [2 7 9 5 6 1 8 4 3]
 [8 9 3 2 1 4 5 6 7]
 [6 5 8 7 2 3 9 1 4]
 [3 1 7 8 4 9 6 2 5]
 [1 2 4 6 9 5 7 3 8]]

验算：求每行每列的和

[45, 45, 45, 45, 45, 45, 45, 45, 45]
[45, 45, 45, 45, 45, 45, 45, 45, 45]
